In [1]:
#hide
import fastbook
fastbook.setup_book()
from fastbook import *

In [7]:
from fastai.vision.all import *
path = untar_data(URLs.IMAGENETTE)

In [22]:
dblock = DataBlock(
    blocks=(ImageBlock(), CategoryBlock()),
    get_items=get_image_files,
    get_y=parent_label,
    item_tfms=Resize(460),
    batch_tfms=aug_transforms(size=224, min_scale=0.75)
)
dls = dblock.dataloaders(path, bs=64)

In [ ]:
dls.show_batch()

In [23]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.633567,2.686260,0.310306,02:42
1,1.280104,4.192402,0.345407,02:44
2,0.965334,1.192987,0.636669,02:44
3,0.750896,0.680817,0.793876,02:41
4,0.606962,0.564866,0.836819,02:42


In [15]:
x, y = dls.one_batch()
x.mean(dim=[0,2,3]), x.std(dim=[0,2,3])

(TensorImage([0.4292, 0.4193, 0.3790], device='cuda:0'),
 TensorImage([0.2785, 0.2721, 0.2929], device='cuda:0'))

In [24]:
def get_dls(bs, size):
    dblock = DataBlock(blocks=(ImageBlock, CategoryBlock),
                   get_items=get_image_files,
                   get_y=parent_label,
                   item_tfms=Resize(460),
                   batch_tfms=[*aug_transforms(size=size, min_scale=0.75),
                               Normalize.from_stats(*imagenet_stats)])
    return dblock.dataloaders(path, bs=bs)

In [25]:
dls = get_dls(64, 224)

In [26]:
model = xresnet50(n_out=dls.c)
learn = Learner(dls, model, loss_func=CrossEntropyLossFlat(), metrics=accuracy)
learn.fit_one_cycle(5, 3e-3)

epoch,train_loss,valid_loss,accuracy,time
0,1.629753,2.276633,0.415235,02:41
1,1.226794,1.728238,0.499627,02:43
2,0.929801,1.926562,0.505228,02:44
3,0.729028,0.670939,0.781927,02:44
4,0.593769,0.546419,0.827110,02:44
